## Install and Initialize the enviromnent

In [ ]:
import gym
import gym_2048
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install gym-2048

     |████████████████████████████████| 1.5 MB 4.3 MB/s 
     |████████████████████████████████| 13.8 MB 5.8 MB/s 
  Created wheel for gym-2048: filename=gym_2048-0.2.6-py3-none-any.whl size=4697 sha256=4c4192ec8973f9f9e9e5ff1aa8dd88f3e1560e72c4cd5ce83e520e37d27a70d9
  Stored in directory: /root/.cache/pip/wheels/68/98/9f/c396b6407bd4c0906c2a6ed5905202cd0d423dd2d6d8db05a2
  Created wheel for gym: filename=gym-0.10.11-py3-none-any.whl size=1588313 sha256=c1540e418e6de69529bdbe864ccc8cdb2fe269fc32d30870c5cdbe5fb0d82d33
  Stored in directory: /root/.cache/pip/wheels/ec/c9/3a/068c5b80305e89c8de8b0a412e67ef2986cbad74895cfb9551
Successfully built gym-2048 gym
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
ERROR: pip's dependency resolver does not cu

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Custom class to be able to define the inital starting state - going to be used for Monte Carlo tree search.

In [ ]:
class NewBase2048Env(gym_2048.Base2048Env):
  def __init__(self, *args):
    gym_2048.Base2048Env.__init__(self, *args)
  def custom_reset(self, starting_board): #new function to allow reset with initial conditions
    self.board = starting_board
    return self.board


## Monte-Carlo Tree Search For 2048


This function takes two arguments:

- `run_simulations`: The number of move sets the algorithm should simulate before making each real move
- `move_simulations`: The number of moves it should simulate in each run

For instance, if `run_simulations` is 50 and `move_simulations` is 10, then before each real move, the algorithm will simulate 50 random sets of 10 moves. Increasing these values will improve performance but increase the time to compute as well.

The algorithm will track the total score accumulated during each simulated run, and will choose the next move to be the one which had the highest average score for all of its simulated runs. For instance, in the example above, the algorithm will test 50 simulated runs of 10 random moves. For each possible move (up, down, left, right), the algorithm will find the average total score accumulated for all random runs which began with that move. Then, it will choose the next move to be the one which had the highest average score accumulated.

In [ ]:
import time

def monte_carlo_2048(run_simulations=100, move_simulations=10, criterion='avg'):
  start = time.time()
  env = NewBase2048Env()

  obs, reward, done, _ = env.step(2)
  moves_made = 0
  total_score = 0
  while not done:
    current_board = env.board.copy() #making a copy of the current board

    future_moves = {0:[], 1:[], 2:[], 3:[]} #list of all ending scores for each possible next move
    for simulated_run in range(run_simulations):
      reward = 0

      simulated_env = NewBase2048Env()
      simulated_env.custom_reset(current_board) #initiate a simulated environment with the current board setup

      action = np.random.choice([0, 1, 2, 3]) #first action
      _, score, lost, _ = simulated_env.step(action)
      reward += score #add the number of points from the move to the total reward from this run
      if lost: #if the game is done, set reward to 0 and break the simulated run
        reward = 0
        
      for simulated_move in range(move_simulations): #additional random actions
        _, score, lost, _ = simulated_env.step(np.random.choice([0, 1, 2, 3]))
        reward += score
        if lost:
          break
          
      future_moves[action].append(reward) #append the total points accumulated from this run to the future_moves dictionary
    
    if criterion=='max':
      #look at max rewards for each possible next move, and choose the one with highest max reward
      best_reward = 0
      best_move = 0
      for key in future_moves:
        if np.max(future_moves[key]) > best_reward:
          best_reward = np.max(future_moves[key])
          best_move = key
    else:
      #look at average rewards for each possible next move, and choose the one with highest average
      best_reward = 0
      best_move = 0
      for key in future_moves:
        if np.mean(future_moves[key]) > best_reward:
          best_reward = np.mean(future_moves[key])
          best_move = key
    
    obs, reward, done, _ = env.step(best_move)
    total_score += reward
    moves_made += 1
    if moves_made % 100 == 0:
      print(f'{moves_made} moves made')
      print(f'Total Score: {total_score}')
      print(f'Current best tile: {np.max(env.board)}')
      print()

      
  print(f'Final Score: {total_score}')
  print(f'Time to play: {time.time()-start}')
  print(f'Best tile achieved: {np.max(env.board)}')
  print()
  env.render()

  return time.time()-start, np.max(env.board), total_score

In [ ]:
def playing_game(n_rounds):
  durations = []
  max_nums = []
  scores = []
  for i in range(n_rounds):
    duration, max_num, score = monte_carlo_2048()
    durations.append(duration)
    max_nums.append(max_num)
    scores.append(score)
  return durations, max_nums, scores

In [ ]:
# Manually run this for 20 times in terms of 100 experiments in total
d, m, s = playing_game(5)

/usr/local/lib/python3.7/dist-packages/gym_2048/env.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  board[tile_locs] = tiles


100 moves made
Total Score: 1048
Current best tile: 128

200 moves made
Total Score: 2616
Current best tile: 256

300 moves made
Total Score: 4576
Current best tile: 512

400 moves made
Total Score: 6204
Current best tile: 512

500 moves made
Total Score: 7692
Current best tile: 512

600 moves made
Total Score: 10852
Current best tile: 1024

700 moves made
Total Score: 12004
Current best tile: 1024

800 moves made
Total Score: 14020
Current best tile: 1024

900 moves made
Total Score: 15616
Current best tile: 1024

1000 moves made
Total Score: 20624
Current best tile: 2048

1100 moves made
Total Score: 22132
Current best tile: 2048

1200 moves made
Total Score: 24272
Current best tile: 2048

1300 moves made
Total Score: 25440
Current best tile: 2048

1400 moves made
Total Score: 26984
Current best tile: 2048

1500 moves made
Total Score: 30100
Current best tile: 2048

1600 moves made
Total Score: 31732
Current best tile: 2048

1700 moves made
Total Score: 33644
Current best tile: 2048


In [ ]:
import pandas as pd
data = {
    'Durations': pd.Series(d),
    'Max_nums': pd.Series(m),
    'Scores': pd.Series(s)
}

In [ ]:
output

,Durations,Max_nums,Scores
0,476.917869,2048,36316
1,410.109159,2048,32216
2,424.113538,2048,32556
3,237.548640,1024,16368
4,670.140238,4096,58440


In [ ]:
def playing_game_small(n_rounds):
  durations = []
  max_nums = []
  scores = []
  for i in range(n_rounds):
    duration, max_num, score = monte_carlo_2048(run_simulations=50, move_simulations=5)
    durations.append(duration)
    max_nums.append(max_num)
    scores.append(score)
  return durations, max_nums, scores

In [ ]:
# Running too many times would cause time-out error because of time limit of colab
# Manually run this for 20 times in terms of 100 experiments in total
d, m, s = playing_game_small(5)

/usr/local/lib/python3.7/dist-packages/gym_2048/env.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  board[tile_locs] = tiles


100 moves made
Total Score: 1096
Current best tile: 128

200 moves made
Total Score: 2652
Current best tile: 256

300 moves made
Total Score: 4592
Current best tile: 512

400 moves made
Total Score: 6196
Current best tile: 512

500 moves made
Total Score: 9252
Current best tile: 1024

600 moves made
Total Score: 10828
Current best tile: 1024

700 moves made
Total Score: 12012
Current best tile: 1024

800 moves made
Total Score: 14148
Current best tile: 1024

900 moves made
Total Score: 15688
Current best tile: 1024

1000 moves made
Total Score: 20668
Current best tile: 2048

1100 moves made
Total Score: 22312
Current best tile: 2048

1200 moves made
Total Score: 24332
Current best tile: 2048

1300 moves made
Total Score: 25944
Current best tile: 2048

Final Score: 26556
Time to play: 203.73986434936523
Best tile achieved: 2048

16 	8 	4 	16
64 	32 	256 	2
8 	512 	2048 	8
4 	32 	4 	2
100 moves made
Total Score: 1020
Current best tile: 128

200 moves made
Total Score: 2620
Current best t

In [15]:
import pandas as pd
data = {
    'Durations': pd.Series(d),
    'Max_nums': pd.Series(m),
    'Scores': pd.Series(s)
}

In [16]:
output

,Durations,Max_nums,Scores
0,203.740105,2048,26556
1,140.058735,1024,16328
2,137.833295,1024,16132
3,252.379892,2048,34456
4,269.830072,2048,36172
